## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Load Dataset

In [2]:
path = "../data/ecom_raw.csv"
df = pd.read_csv(path)

print("Dataset loaded successfully.")
print("Shape: ", df.shape)
df.head()

Dataset loaded successfully.
Shape:  (250000, 13)


,Customer ID,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,Payment Method,Customer Age,Returns,Customer Name,Age,Gender,Churn
0,46251,2020-09-08 09:38:32,Electronics,12,3,740,Credit Card,37,0.0,Christine Hernandez,37,Male,0
1,46251,2022-03-05 12:56:35,Home,468,4,2739,PayPal,37,0.0,Christine Hernandez,37,Male,0
2,46251,2022-05-23 18:18:01,Home,288,2,3196,PayPal,37,0.0,Christine Hernandez,37,Male,0
3,46251,2020-11-12 13:13:29,Clothing,196,1,3509,PayPal,37,0.0,Christine Hernandez,37,Male,0
4,13593,2020-11-27 17:55:11,Home,449,1,3452,Credit Card,49,0.0,James Grant,49,Female,1


# Basic Info

In [3]:
df.info()
df.describe(include='all')
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Customer ID            250000 non-null  int64  
 1   Purchase Date          250000 non-null  object 
 2   Product Category       250000 non-null  object 
 3   Product Price          250000 non-null  int64  
 4   Quantity               250000 non-null  int64  
 5   Total Purchase Amount  250000 non-null  int64  
 6   Payment Method         250000 non-null  object 
 7   Customer Age           250000 non-null  int64  
 8   Returns                202404 non-null  float64
 9   Customer Name          250000 non-null  object 
 10  Age                    250000 non-null  int64  
 11  Gender                 250000 non-null  object 
 12  Churn                  250000 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 24.8+ MB


Customer ID                  0
Purchase Date                0
Product Category             0
Product Price                0
Quantity                     0
Total Purchase Amount        0
Payment Method               0
Customer Age                 0
Returns                  47596
Customer Name                0
Age                          0
Gender                       0
Churn                        0
dtype: int64

# Rename Columns
lowercase and underscores in place of spaces

In [4]:
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
df.head(3)

,customer_id,purchase_date,product_category,product_price,quantity,total_purchase_amount,payment_method,customer_age,returns,customer_name,age,gender,churn
0,46251,2020-09-08 09:38:32,Electronics,12,3,740,Credit Card,37,0.0,Christine Hernandez,37,Male,0
1,46251,2022-03-05 12:56:35,Home,468,4,2739,PayPal,37,0.0,Christine Hernandez,37,Male,0
2,46251,2022-05-23 18:18:01,Home,288,2,3196,PayPal,37,0.0,Christine Hernandez,37,Male,0


## Handling Missing & DUplicates

In [5]:
df = df.drop_duplicates(subset =["customer_id", "purchase_date", "product_category"])

# fill missing numeric cols with median
num_cols = df.select_dtypes(include = [np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# fill missing categorical with mode
cat_cols = df.select_dtypes(exclude=[np.number]).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print ("Missing values handled.")

Missing values handled.


## Fix Data Types

In [11]:
df["purchase_date"] = pd.to_datetime(df["purchase_date"], errors='coerce')

numeric_cols = ["product_price", "quantity", "total_purchase_amount", "customer_age"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=["purchase_date", "product_price", "customer_age"])

print("Data types fixed.")

Data types fixed.


## Feature Engineering

In [12]:
# create month/year columns
df["purchase_month"] = df["purchase_date"].dt.month
df["purchase_year"] = df["purchase_date"].dt.year

# create total if missing
if "total_purchase_amount" not in df.columns or df["total_purchase_amount"].isnull().all():
    df["total_purchase_amount"] = df["product_price"]*df["quantity"]

# Age bins
df["age_group"] = pd.cut(df["customer_age"], bins=[0,18,30,45,60,100], labels=["Teen", "Young Adult", "Adult", "Middle Age", "Senior"])

## Encode Categorical Columns

In [13]:
from sklearn.preprocessing import LabelEncoder

encode_cols = ["product_category", "payment_method", "gender", "returns", "churn"]
le = LabelEncoder()

for col in encode_cols:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

print("Categorical columns encoded.")

Categorical columns encoded.


## Save Processed Data

In [14]:
output_path = "../data/ecommerce_data.csv"
df.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: ../data/ecommerce_data.csv


## Confirm Output

In [15]:
print(df.shape)
df.head()

(250000, 16)


,customer_id,purchase_date,product_category,product_price,quantity,total_purchase_amount,payment_method,customer_age,returns,customer_name,age,gender,churn,age_group,purchase_month,purchase_year
0,46251,2020-09-08 09:38:32,2,12,3,740,1,37,0.0,Christine Hernandez,37,1,0,Adult,9,2020
1,46251,2022-03-05 12:56:35,3,468,4,2739,3,37,0.0,Christine Hernandez,37,1,0,Adult,3,2022
2,46251,2022-05-23 18:18:01,3,288,2,3196,3,37,0.0,Christine Hernandez,37,1,0,Adult,5,2022
3,46251,2020-11-12 13:13:29,1,196,1,3509,3,37,0.0,Christine Hernandez,37,1,0,Adult,11,2020
4,13593,2020-11-27 17:55:11,3,449,1,3452,1,49,0.0,James Grant,49,0,1,Middle Age,11,2020
